## Preparing simulation system

In [24]:
#Libraries and main variables initialized
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis.rms import rmsd
import numpy as np
import urllib.request, json
import os
import re
import nglview as nv
from funcs import get_files_from_git
from funcs import view_nucl
from IPython.display import Markdown, display

#Reloading funcs if changed
from importlib import reload 
import funcs,config
reload(funcs)
reload(config)

%matplotlib inline

from config import folder,descr,pname
display(Markdown(descr))
print("Folder set to: ",folder)
print("Project name: ",pname)



# MD simulations of H3-H4, tails truncated
- AMBER14SB force field
- 50 mM NaCl
- box 2nm


Folder set to:  h3-h4_tm
Project name:  h3-h4_tm


### Donwload and prepare pdb
- We use 1kx5 as in JMB paper https://www.ncbi.nlm.nih.gov/pubmed/26699921
- We use maximim tuncation up to the essential elements (helixes, key arginines, docking domain). But also try not to create unbalanced charges (see figure below).
- The h3-h4_maximally tuncated is H3 residues 43-end(135), H4 residues 23-end(102) (see figure below).
- Due to AMBER ff, we add ACE residues on truncated N-termi using MDanalysis

<img src="cutting.jpg">

In [25]:
%%bash --out output --err error
wget https://files.rcsb.org/view/1KX5.pdb

In [26]:
nucl=mda.Universe("1KX5.pdb")
#For AMBER FF to place ACE on N-truncated end we need to retain one more residue
# CHARMM can handle it differently directly in pdb2gmx
sel_text="(segid A and (resid 44:135 or (resid 43 and name C O CA))) or (segid B and (resid 23:102 or (resid 22 and name C O CA)))"%ids
#and we take first solvation shell
dimer=nucl.select_atoms(sel_text+" or (resname HOH and around 3 %s)"%sel_text)

nucl.select_atoms("segid A and resid 43").residues.resnames='ACE'
nucl.select_atoms("segid A and resid 43 and name CA").atoms.names='CH3'
nucl.select_atoms("segid B and resid 22").residues.resnames='ACE'
nucl.select_atoms("segid B and resid 22 and name CA").atoms.names='CH3'

nucl.trajectory.remarks=[]
nucl.trajectory.compound=[]
nucl.trajectory.header=['H3-H4 dimer radically truncated']

#print(dimer.atoms.names)
dimer.write("h3_h4_tm.pdb")

/opt/miniconda3/envs/moldyn/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:72: UserWarning: Failed to guess the mass for the following atom types: MN
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


In [27]:
#This will currently work only in Jupyter Notebook
nv.show_mdanalysis(dimer,gui=False)

NGLWidget()

### We need to specify force field and path to download it from git
- Here we use AMBER 14SB for Gromacs (Need to check)
- In this repo https://github.com/intbio/gromacs_ff/ we store all the forcefield that we check manually or adjust
- Folder for AMBER 14SB ff is amber14sb_parmbsc1.ff 
- Folder for AMBER 14SB ff with CUFIX is amber14sb_parmbsc1_cufix.ff   
- We need to specify a GitHub API URL for it below

In [28]:
#Set URL here
#Intentianally commented out to make an informed choice
#ff_name='amber14sb_parmbsc1'
#ff_name='amber14sb_parmbsc1_cufix'
ffurl="https://api.github.com/repos/intbio/gromacs_ff/contents/%s.ff"%ff_name
print("FF set to ",ff_name)

FF set to  amber14sb_parmbsc1


In [ ]:
## Download protocols right away!

In [ ]:
%%bash
rm -rf MDProtocols

In [ ]:
#Set protocols URL
prot_url="https://api.github.com/repos/intbio/gmx_protocols/contents/amber"

In [ ]:
get_files_from_git(prot_url,'MDProtocols/')

### Prepare directories, download forcefield

In [29]:
%%bash
#clean GMX_system
rm -rf GMX_system
mkdir -p GMX_system

In [30]:
get_files_from_git(ffurl,'GMX_system/%s.ff'%ff_name)

### Let's start with Gromacs commands

- manual for 2018 version is here http://manual.gromacs.org/documentation/current/manual-2018.3.pdf 
- Command line reference http://manual.gromacs.org/documentation/current/user-guide/cmdline.html


### PDB2GMX
- Prepare initial topology (.top), coordinates (.gro) and restrain files .
- http://manual.gromacs.org/documentation/current/onlinehelp/gmx-pdb2gmx.html

In [31]:
%%bash -s "$ff_name" --err err
# PDB2GMX
# Make an init gro file and topology
# Force filed AMBER99BSC1 !!!!!!!!!
# Water TIP3P
# Termini !IMPORTANT! on N-end we place  ACE termini on C-end we place charged group.
# NOTE: The AMBER force fields have unique forms for the terminal residues,
# and these are incompatible with the -ter mechanism.~
# You need to prefix your N- or C-terminal residue names with “N” or “C” respectively
# to use these forms, making sure you preserve the format of the coordinate file.
# Alternatively, use named terminating residues (e.g. ACE, NME).

#TODO we need to understnt ter in amber!!! and fix
cd GMX_system
gmx pdb2gmx -f ../h3_h4_tm.pdb -o init.pdb -p topol.top -i posre.itp -water tip3p -ff $1


Using the Amber14sb_parmbsc1 force field in directory ./amber14sb_parmbsc1.ff

Reading ../h3_h4_tm.pdb...
Read 'MDANALYSIS FRAME 0: Created by PDBWriter', 1394 atoms
Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.
There are 2 chains and 0 blocks of water and 174 residues with 1394 atoms

  chain  #res #atoms
  1 'A'    93    753  
  2 'B'    81    641  

Reading residue database... (amber14sb_parmbsc1)
Processing chain 1 'A' (753 atoms, 93 residues)
Identified residue ACE43 as a starting terminus.
Identified residue ALA135 as a ending terminus.
Checking for duplicate atoms....
Generating any missing hydrogen atoms and/or adding termini.
Now there are 93 residues with 1546 atoms
Chain time...
Processing chain 2 'B' (641 atoms, 81 residues)
Identified residue ACE22 as a starting terminus.
Identified residue GLY102 as a ending terminus.
Checking for duplicate atoms....
Generating any missing hydrogen atoms and/or adding termini.
Now there are 81 re

In [32]:
# Get total charge
for line in err.split('\n'):
    if 'Total charge in system' in line:
        charge=float(line.split()[-2])
print('Total charge is %.3f'%charge)

Total charge is 14.000


### Fixing posre.itp files
see this link for details
http://mdsquad.wikia.com/wiki/Change_position_restraint_force_constant_in_MDP

In [33]:
%%bash
cd GMX_system/
for i in posre_*; do
sed -i.bak 's/\(.*\)1000  1000  1000/\1 POSRES_FC POSRES_FC POSRES_FC/g' $i
#check if repaced
tail -n 1 $i
done
rm *.bak



  1546     1   POSRES_FC POSRES_FC POSRES_FC
  1321     1   POSRES_FC POSRES_FC POSRES_FC


### Solvating a molecule
- 2 nm to the border in a cubic box.
- http://manual.gromacs.org/documentation/current/onlinehelp/gmx-editconf.html
- http://manual.gromacs.org/documentation/current/onlinehelp/gmx-solvate.html

In [34]:
%%bash --err err
cd GMX_system
gmx editconf -bt octahedron -d 2 -c -f init.pdb -o init_box.pdb
gmx solvate -cp init_box.pdb -cs spc216.gro -o init_solv.pdb -p topol.top

Read 2867 atoms
Volume: 2101.66 nm^3, corresponds to roughly 945700 electrons
No velocities found
    system size :  3.951  5.406  4.435 (nm)
    diameter    :  6.239               (nm)
    center      :  6.284  9.788  1.201 (nm)
    box vectors : 10.595 18.117 10.949 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :2101.66               (nm^3)
    shift       : -1.165 -2.548  2.979 (nm)
new center      :  5.119  7.240  4.180 (nm)
new box vectors : 10.239 10.239 10.239 (nm)
new box angles  :  70.53 109.47  70.53 (degrees)
new box volume  : 826.23               (nm^3)

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may

### Determine the number of ions

- concentration is 150 mM NaCl
- we will calculate the number of waters and calculate the number of ions to add.

In [35]:
for line in err.split('\n'):
    if 'Number of solvent molecules:' in line:
        n_sol=int(line.split()[-1])
print('Amount of solvent molecules: %d'%n_sol)

#Set concentration in mol/liter
conc=0.150 
#Water 18 g/mol. in 1 liter 1000/18 mol
# for every 1000/18 water molecules we need conc of NaCl ion pairs.

n_ions=round(conc*(n_sol/(1000/18)))

if charge>=0:
    n_pos=int(n_ions)
    n_neg=int(n_ions+charge)
else:
    n_pos=int(n_ions-charge)
    n_neg=int(n_ions)
print('Amount of Na %d \nAmount of Cl %d'%(n_pos,n_neg))

Amount of solvent molecules: 26049
Amount of Na 70 
Amount of Cl 84


### Adding ions
- http://manual.gromacs.org/documentation/current/onlinehelp/gmx-grompp.html
- http://manual.gromacs.org/documentation/current/onlinehelp/gmx-genion.html

In [36]:
%%bash -s "$n_pos" "$n_neg"
cd GMX_system
gmx grompp -f ../MDProtocols/ions.mdp -c init_solv.pdb -p topol.top -o ions.tpr -maxwarn 1
gmx genion -s ions.tpr -o init_solv_ions.pdb -p topol.top -noneutral -pname NA -nname CL -np $1 -nn $2 <<!
13
!


++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
J. S. Hub, B. L. de Groot, H. Grubmueller, G. Groenhof
Quantifying Artifacts in Ewald Simulations of Inhomogeneous Systems with a Net
Charge
J. Chem. Theory Comput. 10 (2014) pp. 381-393
-------- -------- --- Thank You --- -------- --------

Analysing residue names:
There are:   174    Protein residues
There are: 26049      Water residues
Analysing Protein...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 96x96x96, spacing 0.107 0.107 0.107
This run will generate roughly 6 Mb of data
Will try to add 70 NA ions and 84 CL ions.
Select a continuous group of solvent molecules
Selected 13: 'SOL'

Processing topology
Replacing 154 solute molecules in topology file (topol.top)  by 70 NA and 84 CL ions.


                      :-) GROMACS - gmx grompp, 2018.3 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS de

### Making index file

http://manual.gromacs.org/documentation/2018/onlinehelp/gmx-make_ndx.html

We also need index that will have a group for non water and ions components

In [2]:
%%bash
#The group need to be adjusted!!!
cd GMX_system
gmx make_ndx -f init_solv_ions.pdb -o index.ndx <<!
!18
q
!

Going to read 0 old index file(s)
Analysing residue names:
There are:   174    Protein residues
There are: 25895      Water residues
There are:   154        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 80706 atoms
  1 Protein             :  2867 atoms
  2 Protein-H           :  1394 atoms
  3 C-alpha             :   172 atoms
  4 Backbone            :   518 atoms
  5 MainChain           :   694 atoms
  6 MainChain+Cb        :   855 atoms
  7 MainChain+H         :   863 atoms
  8 SideChain           :  2004 atoms
  9 SideChain-H         :   700 atoms
 10 Prot-Masses         :  2867 atoms
 11 non-Protein         : 77839 atoms
 12 Water               : 77685 atoms
 13 SOL                 : 77685 atoms
 14 non-Water           :  3021 atoms
 15 Ion                 :   154 atoms
 16 NA                  :    70 atoms
 17 CL                  :    84 atoms
 18 Water_and_ions      : 77839 atom

                     :-) GROMACS - gmx make_ndx, 2018.3 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS d

## Making X-ray initial protein file to serve as a reference during analysis

A problem is that gromacs looses info about chain names - which is not good at all and will complicate downstream analysis. So we need to retain it.
init.pdb - still has the correct chain IDs, and since water and ions are added below it can serve for downstream analysis as reference structure.

In [40]:
%%bash 

#--out out
#gmx trjconv -f GMX_system/init_solv.pdb -s GMX_system/ions.tpr -o GMX_system/prot_ref.pdb << !
#1
#!

#Alternative
#gmx editconf -label R E -f GMX_system/init_solv_ions.pdb -n GMX_system/index.ndx -o GMX_system/prot_ref.pdb << !
#1
#!



In [39]:
%%bash
#cp GMX_system/init.pdb GMX_system/prot_ref.pdb

cd GMX_system
gmx make_ndx -f init.pdb -o index_ref.ndx <<!
q
!
cd ..
#gmx trjconv -f GMX_system/init_solv.pdb -s GMX_system/ions.tpr -o GMX_system/bb_ref.pdb << !
#4
#!
#gmx trjconv -f GMX_system/init_solv.pdb -s GMX_system/ions.tpr -o GMX_system/bb-h_ref.pdb << !
#7
#!

#Alternative
gmx editconf -f GMX_system/init.pdb -n GMX_system/index_ref.ndx -o GMX_system/prot_ref.pdb << !
15
!
gmx editconf -f GMX_system/init.pdb -n GMX_system/index_ref.ndx -o GMX_system/bb_ref.pdb << !
4
!
gmx editconf -f GMX_system/init.pdb -n GMX_system/index_ref.ndx -o GMX_system/bb-h_ref.pdb << !
7
!



Going to read 0 old index file(s)
Analysing residue names:
There are:   174    Protein residues
Analysing Protein...

  0 System              :  2867 atoms
  1 Protein             :  2867 atoms
  2 Protein-H           :  1394 atoms
  3 C-alpha             :   172 atoms
  4 Backbone            :   518 atoms
  5 MainChain           :   694 atoms
  6 MainChain+Cb        :   855 atoms
  7 MainChain+H         :   863 atoms
  8 SideChain           :  2004 atoms
  9 SideChain-H         :   700 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr         'splitres' nr   'l': list residues
 't': atom type  '|': or   'keep' nr        'splitat' nr    'h': help
 'r': residue              'res' nr         'chain' char
 "name": group             'case': case sensitive           'q': save and quit
 'ri': residue index

> 
Read 2867 atoms
Volume: 2101.66 nm^3, corresponds to roughly 945700 electrons
No velocities found
Selected 4: 'B

                     :-) GROMACS - gmx make_ndx, 2018.3 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS d

As a result in GMX_system directory we have a prepared system for minimization and further simulations.